# Text Analytics Exploration

## Import Data

In [ ]:
report_url = "http://hansardpublic.parliament.sa.gov.au/Pages/HansardResult.aspx#/docid/HANSARD-10-25756"
report_text = "When I tried to get a copy of the commissioner's report after being tabled, why was I basically told that there was a very limited— The PRESIDENT: This is a matter of personal explanation in a supplementary. Just please, the Hon. Mr Wortley, ask your supplementary. The Hon. R.P. WORTLEY: Why weren't all members of parliament given a copy of the royal commission's report? The Hon. D.W. Ridgway: But you told us before you never read reports. The Hon. R.I. LUCAS (Treasurer) (15:26): Mr President, I won't go down that particular path, as delicious as that interjection might have been in relation to the Hon. Mr Wortley saying he couldn't trust himself to read his own reports. I don't know why the Hon. Mr Wortley was unable to get a copy of the royal commission report. It was certainly publicly available. If it pleases the member, I will see whether there is not a spare copy somewhere. If we do find a spare copy and give it to him, I will be asking questions afterwards of the honourable member just to make sure he did read it. The Hon. D.W. Ridgway: Do you want it delivered to Scuzzi or something more convenient for you? The PRESIDENT: Are you finished, the Hon. Mr Ridgway? The Hon. R.P. WORTLEY: You just worry about our trade exports, mate, for the state. The PRESIDENT: The Hon. Mr Wortley, I am waiting patiently here to give you the call for your question. Have you finished your private conversation with the Hon. Mr Ridgway? Yes? The Hon. Mr Wortley."
report_title = "Murray-Darling Basin Royal Commission"

In [ ]:
# Import data from spreadsheet
import pandas as pd
import numpy as np

data = pd.read_excel ("..\\data\\HANSARDfullDataset.xlsx", sheet_name="text")
df = pd.DataFrame(data, columns= ['hansardID','text'])
df = df.astype({"hansardID":'str', "text":'str'}) 

#df.dtypes 
df.head(2)

In [ ]:
#hansardFilesInfo = pd.read_excel ("..\\data\\HANSARDfullDataset.xlsx", sheet_name="HANSARDFilesInfo")
#hansardFilesInfo = pd.DataFrame(hansardFilesInfo, columns= ['FileName','URL'])
#hansardFilesInfo = hansardFilesInfo.astype({"FileName":'str', "URL":'str'}) 
#hansardFilesInfo.head(2)

In [ ]:
#header = pd.read_excel ("..\\data\\HANSARDfullDataset.xlsx", sheet_name="header")
#header = pd.DataFrame(header)
#header.head(2)

In [ ]:
#bill = pd.read_excel ("..\\data\\HANSARDfullDataset.xlsx", sheet_name="bill")
#bill = pd.DataFrame(bill, columns= ['question','bname'])
#bill.head(2)

In [ ]:
# Group text into one document
grouped_text = df.groupby('hansardID')['text'].agg(lambda col: '. '.join(col))
grouped_text_df = pd.DataFrame(grouped_text, columns= ['text'])
grouped_text_df.head(5)

In [ ]:
grouped_text.iloc[4].replace("..",".")

## Basic Text Analytics

### Word Count

In [ ]:
import re

# Count number of words in sentence using regex
grouped_text_df['WordCount'] = grouped_text_df.apply(lambda x: len(re.findall(r'\w+', x.text)), axis=1)
grouped_text_df.head(2)

### Length of Sentences

In [ ]:
from statistics import median

all_text = df['text'].agg(lambda col: ''.join(col))
all_text = ". ".join(all_text)

# Average number of words in a sentence
parts = [len(l.split()) for l in re.split(r'[?!.:] ', ' '.join(all_text)) if l.strip()]
print("Average = ", sum(parts)/len(parts))

# Median number of words in a sentence
print("Median = ", median(parts))
print("Min = ", min(parts))
print("Max = ", max(parts))
print("Q1 = ", np.percentile(parts,25))
print("Q3 = ", np.percentile(parts,75))

## Document Summarisation

Articles and libraries to look into further: 
* https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
* https://stackabuse.com/text-summarization-with-nltk-in-python/
* https://github.com/alanbuxton/PyTeaserPython3
* https://github.com/abisee/pointer-generator
* https://github.com/DerwenAI/pytextrank
* https://github.com/tensorflow/models/tree/master/research/textsum
* https://radimrehurek.com/gensim/models/lsimodel.html
* https://towardsdatascience.com/text-summarization-in-python-76c0a41f0dc4 (additional links to articles at the end)

### Feature Base Text Summarisation

In [ ]:
# The feature base model extracts the features of the sentence, then evaluate its importance
# Feature base text summarization by TextTeaser
#from pyteaser import SummarizeUrl
#url = 'http://www.huffingtonpost.com/2013/11/22/twitter-forward-secrecy_n_4326599.html'
#summaries = SummarizeUrl(url)
#print summaries

In [ ]:
# TextTeasor - automatic summarization algorithm that combines the power of natural language processing and machine learning
#from textteaser import TextTeaser
#tt = TextTeaser()
#tt.summarize(title, text)

### Topic Model Summary

In [ ]:
# Topic Model summarisation
from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
model = LsiModel(common_corpus, id2word=common_dictionary)
vectorized_corpus = model[common_corpus]
#print(vectorized_corpus)
#model.print_topics(1)

### Truncated Sentences

In [ ]:
def smart_truncate(content, length=200, suffix='...'):
    if len(content) <= length:
        return content
    else:
        return ' '.join(content[:length+1].split(' ')[0:-1]) + suffix

smart_truncate(report_text)

In [ ]:
grouped_text_df['Truncated'] = grouped_text_df.apply(lambda x: smart_truncate(x.text), axis=1)
grouped_text_df.head(5)

### Extract Keywords

In [ ]:
# https://rare-technologies.com/text-summarization-with-gensim/
#
from gensim.summarization import keywords

def get_keywords(text):
    keyword_list = keywords(text, split=True, lemmatize=True, deacc=True)
    return ', '.join(keyword_list[:10])

grouped_text_df['KeyWords'] = grouped_text_df.apply(lambda x: 
                                                    get_keywords(x.text), 
                                                    axis=1)
grouped_text_df.head(5)

In [ ]:
get_keywords(report_text)

### PyTextRank

In [ ]:
# https://github.com/DerwenAI/pytextrank
# https://github.com/DerwenAI/pytextrank/blob/master/example.ipynb
# Requires JSON input

### TextRank Summary

In [ ]:
from gensim.summarization.summarizer import summarize

#  TextRank summarization with default parameters
grouped_text_df['TextRank'] = grouped_text_df.apply(lambda x: 
                                                    summarize(x.text).replace("\n"," ").replace("..","."), 
                                                    axis=1)

#  TextRank summarization with no more than 50 words for the summary
grouped_text_df['TextRank50'] = grouped_text_df.apply(lambda x: 
                                                      summarize(x.text, word_count = 50).replace("\n"," ").replace("..","."), 
                                                      axis=1)
grouped_text_df.head(5)

In [ ]:
print(summarize(report_text)) #  TextRank summarization

In [ ]:
print(summarize(report_text, word_count = 50)) #  TextRank summarization - no more than 50 words for summary

In [ ]:
print(summarize(report_text, ratio = 0.2)) #  TextRank summarization - use no more than 20% of original text for summary

In [ ]:
summarize(grouped_text.iloc[4], word_count = 50)

In [ ]:
summarize(grouped_text.iloc[2], word_count = 50)

In [ ]:
def generate_text_rank_summary(text):
    
    if len(text) <= 200:
        return text
    
    sentences = []    
    text_sentences = re.split(r'[?!.] ', text)
    
    for sentence in text_sentences:
        processed = sentence.replace("[^a-zA-Z]", " ")
        word_count = len(re.findall(r'\w+', processed)) 
        if word_count > 1: # Include sentences with more than one word
            sentences.append(processed)
    
    summary = summarize('. '.join(sentences))             
    return summary.replace("\n"," ").replace("..",".")


In [ ]:
#  TextRank summarisation with no more than 50 words for the summary
grouped_text_df['TextRankProcessed'] = grouped_text_df.apply(lambda x: generate_text_rank_summary(x.text), axis=1)
grouped_text_df.head(5)

In [ ]:
generate_text_rank_summary(grouped_text.iloc[4])

In [ ]:
generate_text_rank_summary(grouped_text.iloc[2])

In [ ]:
generate_text_rank_summary(report_text)

In [ ]:
# https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70
# Approach uses TextRank algorithm
# TextRank does not rely on any previous training data and can work with any arbitrary piece of text. 
# TextRank is a general purpose graph-based ranking algorithm for NLP

# Import all necessary libraries
import os
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer # is based on The Porter Stemming Algorithm
import numpy as np
import networkx as nx
import re 
import string

nltk.download('stopwords')
nltk.download('wordnet')

# Generate clean sentences
def create_long_sentences(article, output=True):
    sentences = []
    
    for sentence in article:
        word_count = len(re.findall(r'\w+', sentence)) 
        if word_count > 4: # Include sentences with more than four words
            sentences.append(sentence.split(" "))
        if output: 
            print(sentence, ": words = ", word_count)
        
    return sentences   


# Process word by converting to lowercase, removing punctuation, lemmatising, and stemming. 
# https://medium.com/@pemagrg/pre-processing-text-in-python-ad13ea544dae
def process_word(word):
    
    wordnet_lemmatizer = WordNetLemmatizer()
    snowball_stemmer = SnowballStemmer('english')
    
    word = word.lower() # convert to lowercase
    word = word.translate(str.maketrans('', '', string.punctuation)) # remove punctuation 
    word = wordnet_lemmatizer.lemmatize(word) # lemmatise words
    word = snowball_stemmer.stem(word) # stemming
    
    return word


# Process sentence by converting to lowercase, removing punctuation, lemmatising, stemming and removing stopwords. 
# Sentence must be a list of words that make up the sentence
# https://medium.com/@pemagrg/pre-processing-text-in-python-ad13ea544dae
def process_sentence(sentence):
    
    # Process words in sentence
    processed_sentence = [process_word(word) for word in sentence]
    
    # Remove stopwords
    stop_words = stopwords.words('english')
    processed_sentence = [word for word in processed_sentence if word not in stop_words]
    
    return processed_sentence


# Similarity matrix
def sentence_similarity(sent1, sent2):
        
    all_words = list(set(sent1 + sent2)) 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        #if w in stopwords:
        #    continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        #if w in stopwords:
        #    continue
        vector2[all_words.index(w)] += 1
    
    # Vectors must be non-zero to calculate cosine similarity
    if vector1 == 0 or vector2 == 0:
        return 1
 
    return 1 - cosine_distance(vector1, vector2)


def build_similarity_matrix(sentences):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    
    processed_sentences = [process_sentence(sentence) for sentence in sentences]
     
    for idx1 in range(len(processed_sentences)):
        for idx2 in range(len(processed_sentences)):
            if idx1 == idx2: # ignore if both are same sentences
                continue 
            sentences_similarity = sentence_similarity(processed_sentences[idx1], processed_sentences[idx2])
            similarity_matrix[idx1][idx2] = sentences_similarity

    return similarity_matrix


# Generate Summary Method
def generate_summary_from_file(file_name, top_n=5, output=True):
    # Read text and split it into sentences
    file = open(file_name, "r")
    file_data = file.readlines()
    sentences = re.split(r'[?!.] ', file_data[0])
    
    long_sentences = create_long_sentences(sentences, output)
    return generate_summary(long_sentences, top_n, output)


def generate_summary_from_text(text, top_n=5, output=True):    
    
    if len(text) <= 200:
        # If text is short, return the original text rather than a summary
        return text
    else:  
        # Read text and split it into sentences
        sentences = re.split(r'[?!.] ', text)
        long_sentences = create_long_sentences(sentences, output)
        return generate_summary(long_sentences, top_n, output)  
    

def generate_summary(sentences, top_n=5, output=True):
    summarize_text = []
    
    # Generate Similarly Matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences)

    # Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    try: 
        # https://stackoverflow.com/questions/13040548/networkx-differences-between-pagerank-pagerank-numpy-and-pagerank-scipy
        scores = nx.pagerank(sentence_similarity_graph) # The eigenvector calculation is done by the power iteration method and has no guarantee of convergence
        #scores = nx.pagerank_numpy(sentence_similarity_graph) # The eigenvector calculation uses NumPy’s interface to the LAPACK eigenvalue solvers. This will be the fastest and most accurate for small graphs.
        #scores = nx.pagerank_scipy(sentence_similarity_graph) # SciPy sparse-matrix implementation of the power-method
    except nx.NetworkXError:
        print ("NetworkXError")
        return "NetworkXError" # generate_text_rank_summary(text)
    except nx.PowerIterationFailedConvergence:
        print ("PowerIterationFailedConvergence")
        return "PowerIterationFailedConvergence" # generate_text_rank_summary(text)
    
    # Sort the rank and pick top sentences
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    if output:
        print("Indexes of top ranked_sentence order are ", ranked_sentences)    

    if len(ranked_sentences) < top_n:
        top_n = len(ranked_sentences)
        
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentences[i][1]))

    # Output the summary text        
    return '. '.join(summarize_text).replace("..",".")


In [ ]:
sentence = ['The', 'company', 'will', 'provide', 'AI', 'development', 'tools', 'and', 'Azure', 'AI', 'services', 'such', 'as', 'Microsoft', 'Cognitive', 'Services,', 'Bot', 'Services', 'and', 'Azure', 'Machine', 'Learning.According', 'to', 'Manish', 'Prakash,', 'Country', 'General', 'Manager-PS,', 'Health', 'and', 'Education,', 'Microsoft', 'India,', 'said,', '"With', 'AI', 'being', 'the', 'defining', 'technology', 'of', 'our', 'time,', 'it', 'is', 'transforming', 'lives', 'and', 'industry', 'and', 'the', 'jobs', 'of', 'tomorrow', 'will', 'require', 'a', 'different', 'skillset']
processed_sentence = process_sentence(sentence)
print(processed_sentence)

In [ ]:
# let's begin
generate_summary_from_file(os.getcwd() + "\\test.txt", 3, False)

In [ ]:
generate_summary_from_text(grouped_text.iloc[4], 3, False)

In [ ]:
generate_summary_from_text(grouped_text.iloc[2], 3, True)

In [ ]:
generate_summary_from_text(report_text, 3, False)

In [ ]:
# Iterate over all text in data frame and add summary
grouped_text_df['TextRankCustom'] = grouped_text_df.apply(lambda x: generate_summary_from_text(x.text, 3, False), axis=1)

In [ ]:
grouped_text_df.head(5)

### Excel Output of Document Summaries

In [ ]:
grouped_text_df.to_excel('.\\DocumentSummary.xlsx', sheet_name='TextRank', index=True)